# Linear Regression - Apache Spark

This example contains a demo of using Spark's Linear Regression algorithm along with the Vertica database. 

Old Faithful is a geyser that sits in Yellowstone National Park. Using Linear Regression we want to train a model that can predict how long an eruption will be based off the time taken between eruptions.

## Spark Setup

First we start with the basics of setting up Spark to work with Vertica. To do this we need to create a Spark Context that has the Spark Connector passed through it as a configuration option. 

In [ ]:
# Get Connector JAR name
import glob
import os

files = glob.glob("/spark-connector/connector/target/scala-2.12/spark-vertica-connector-assembly-*")
os.environ["CONNECTOR_JAR"] = files[0]
print(os.environ["CONNECTOR_JAR"])

In [ ]:
# Create the Spark session and context
from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .config("spark.master", "spark://spark:7077")
    .config("spark.driver.memory", "2G")
    .config("spark.executor.memory", "1G")
    .config("spark.jars", os.environ["CONNECTOR_JAR"])
    .getOrCreate())
sc = spark.sparkContext

## Import Data

Our Faithful dataset has been randomly split up into two. One for training the model and one for testing it. Both sets are stored in a local .csv, so let's open them and copy them. We can then write each one to Vertica to their respective tables "faithful_training" and "faithful_testing."\
Normally when performing training and testing in ML, we start one with one full dataset and use a function that randomly splits up the dataset. In our case however, we want the datasets the same across Vertica examples to be consistent with our training and results.

In [ ]:
# Load the training set from a CSV file into a dataframe and show some if its contents
df = spark.read.options(header="true", inferschema="true").csv("/spark-connector/examples/jupyter/data/faithful_training.csv")
df.printSchema()
df.show() # So that we can see a bit of what our training dataset looks like and what the schema is

# Write the training data into a table in Vertica
df.write.mode("overwrite").format("com.vertica.spark.datasource.VerticaSource").options(
    host="vertica",
    user="dbadmin",
    password="",
    db="docker",
    table="faithful_training",
    staging_fs_url="webhdfs://hdfs:50070/linearregression").save()

# Do the same write for the testing set
df = spark.read.options(header="true", inferschema="true").csv("/spark-connector/examples/jupyter/data/faithful_testing.csv")
df.write.mode("overwrite").format("com.vertica.spark.datasource.VerticaSource").options(
    host="vertica",
    user="dbadmin",
    password="",
    db="docker",
    table="faithful_testing",
    staging_fs_url="webhdfs://hdfs:50070/linearregression").save()

print("Data of the Old Faithful geyser in Yellowstone National Park.")
print("eruptions = duration of eruption \nwaiting = time between eruptions")

## Read Data

Now that our data is saved in Vertica. We can read from both tables and store them once again in a Spark DF for processing using PySpark's ML toolkit.

In [ ]:
# Read our training data from Vertica into a Spark dataframe
df_training = spark.read.load(format="com.vertica.spark.datasource.VerticaSource",
    host="vertica",
    user="dbadmin",
    password="",
    db="docker",
    table="faithful_training",
    staging_fs_url="webhdfs://hdfs:50070/linearregression")

# Read our testing data from Vertica into a Spark dataframe
df_testing = spark.read.load(format="com.vertica.spark.datasource.VerticaSource",
    host="vertica",
    user="dbadmin",
    password="",
    db="docker",
    table="faithful_testing",
    staging_fs_url="webhdfs://hdfs:50070/linearregression")

## Select Features

Linear Regression analyzes the relationship between an independant and dependant variable using a line of best fit. The dependant variable (eruptions) is what we are trying to predict, whereas the independant variables consists of our features that we are using to make our model. In this case we just have the one variable "waiting", and this will compose our features array.

In [ ]:
# Import Spark's ML Regression tool
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

# Spark's Linear Regression tool requires an array of the features we want to use. Since we only have one in this case, we add "waiting"
featureassembler = VectorAssembler(inputCols = ["waiting"], outputCol = "features")

# Show our new table with a features column added. We are also going to do the same with the testing table so we can compare our results later.
df_testing = featureassembler.transform(df_testing)
df_training = featureassembler.transform(df_training)
df_training.show()

## Train Model

We can now train our model against our training set. We specify our new features column as well as our target "eruptions."

In [ ]:
# Create our model using the features to predict eruption duration and fit it against our training set
lr = LinearRegression(maxIter=10, regParam=0.01, elasticNetParam=1, featuresCol="features", labelCol="eruptions")
lr = lr.fit(df_training)

## Test Model & Results

Our test data comprises of the missing eruption points in "faithful_training." We are now going to use this dataset and compare it against our model to see how the predictions stack up. From there we also want to evaluate the model and see some statistics to show how our algorithm holds up.

In [ ]:
testing_predictions = lr.transform(df_testing)
testing_predictions.select("id", "eruptions","prediction","features").show(20)

test_result = lr.evaluate(df_testing)
print("R Squared (R^2) on test data = %g" % test_result.r2)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

**R Squared** is a calculation that provides us with a way of quantifying the relationship between our variables. \
It is a percentage, with 100% being a 1:1 relationship between our axis.

**RMSE** is the average deviation of the dependant variables to the regression line. \
As such, a value closer to 0 means there is less deviation and therefore less error. Given our unit dimensions (minutes) An RMSE under 0.5 means the model can likely predict values accurately.